In [ ]:
import boto3
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
import os
import pickle

# this is using login to the API via OAuth which enables private data for your own channel

credentials = None

# token.pickle stores the user's credentials from previously successful logins
if os.path.exists('token.pickle'):
    print('Loading credentials from file...')
    with open('token.pickle', 'rb') as token:
        credentials = pickle.load(token)

if not credentials or not credentials.valid:
    if credentials and credentials.expired and credentials.refresh_token:
        print('Refreshing Access Token...')
        credentials.refresh(Request())
    else:
        print('Fetching new tokens...')
        flow = InstalledAppFlow.from_client_secrets_file(
            'client_secrets.json',
            scopes=[
                'https://www.googleapis.com/auth/youtube.readonly',
                'https://www.googleapis.com/auth/yt-analytics.readonly'
                ]
            )
        
        flow.run_local_server(port=8080, prompt='consent')
        # you receive access and refresh tokens
        # refresh token is to get new acccess tokens
        credentials = flow.credentials

        #save
        with open('token.pickle', 'wb') as f:
            print('Saving credentials for future use...')
            pickle.dump(credentials, f)

# Create YouTube API object
youtubeReporting = build('youtubereporting', 'v1', credentials=credentials)

In [ ]:
# Initialize the DynamoDB client
dynamodb = boto3.client('dynamodb')

# Define the specifications for each table
tables = [
    {
        'TableName': 'channel_basic_a2',
        'KeySchema': [
            {'AttributeName': 'composite_key', 'KeyType': 'HASH'},  # Partition key
            {'AttributeName': 'createTime', 'KeyType': 'RANGE'}  # Sort key
        ],
        'AttributeDefinitions': [
            {'AttributeName': 'composite_key', 'AttributeType': 'S'},
            {'AttributeName': 'createTime', 'AttributeType': 'S'}
        ],
        'BillingMode': 'PAY_PER_REQUEST'  # Use on-demand billing mode
    },
    {
        'TableName': 'channel_combined_a2',
        'KeySchema': [
            {'AttributeName': 'composite_key', 'KeyType': 'HASH'},  # Partition key
            {'AttributeName': 'createTime', 'KeyType': 'RANGE'}  # Sort key
        ],
        'AttributeDefinitions': [
            {'AttributeName': 'composite_key', 'AttributeType': 'S'},
            {'AttributeName': 'createTime', 'AttributeType': 'S'}
        ],
        'BillingMode': 'PAY_PER_REQUEST'  # Use on-demand billing mode
    },
    {
        'TableName': 'channel_demographics_a1',
        'KeySchema': [
            {'AttributeName': 'composite_key', 'KeyType': 'HASH'},  # Partition key
            {'AttributeName': 'createTime', 'KeyType': 'RANGE'}  # Sort key
        ],
        'AttributeDefinitions': [
            {'AttributeName': 'composite_key', 'AttributeType': 'S'},
            {'AttributeName': 'createTime', 'AttributeType': 'S'}
        ],
        'BillingMode': 'PAY_PER_REQUEST'  # Use on-demand billing mode
    },
    {
        'TableName': 'channel_sharing_service_a1',
        'KeySchema': [
            {'AttributeName': 'composite_key', 'KeyType': 'HASH'},  # Partition key
            {'AttributeName': 'createTime', 'KeyType': 'RANGE'}  # Sort key
        ],
        'AttributeDefinitions': [
            {'AttributeName': 'composite_key', 'AttributeType': 'S'},
            {'AttributeName': 'createTime', 'AttributeType': 'S'}
        ],
        'BillingMode': 'PAY_PER_REQUEST'  # Use on-demand billing mode
    },
    {
        'TableName': 'reports',
        'KeySchema': [
            {'AttributeName': 'id', 'KeyType': 'HASH'}  # Partition key
        ],
        'AttributeDefinitions': [
            {'AttributeName': 'id', 'AttributeType': 'S'}
        ],
        'BillingMode': 'PAY_PER_REQUEST'  # Use on-demand billing mode
    }
]

# Create each DynamoDB table
for table in tables:
    response = dynamodb.create_table(**table)
    print(f"Created table: {response['TableDescription']['TableName']}")